In [ ]:
# ruff: noqa

In [ ]:
import dvc.api
import pandas as pd

pd.set_option("display.max_columns", 500)

metrics = dvc.api.metrics_show("../output/energy_metrics.json")
metrics

In [ ]:
exps = dvc.api.exp_show(
    param_deps=False, revs="06f2645a7524f2f68c89f917376212a58273b032"
)

In [ ]:
# tags = dvc.api.scm.all_tags()
# exps = dvc.api.exp_show(revs=tags)
df = pd.DataFrame(exps)
df.head()

In [ ]:
energy_metrics = [col for col in df.columns if "energy_metrics.json" in col]
bikes_metrics = [col for col in df.columns if "bikes_metrics.json:" in col]
lgbm = [col for col in df.columns if "lgbm.kwargs" in col]
params_col = [col for col in df.columns if "bikes.selected" in col]
df = pd.DataFrame(exps, columns=bikes_metrics + params_col + ["Experiment"])
df = df.dropna()
# strip the prefix in the col name until : (but not for Created col)
df.columns = [col.split(":")[1] if ":" in col else col for col in df.columns]
df = df.iloc[:]
df.head()

In [ ]:
df = df.drop(columns=["avg_fit_time", "avg_pred_time"])
df.head()

In [ ]:
df["score"] = (
    0.5 * df["pinball_loss.mean"]
    + 0.25 * df["interval_score_50.mean"]
    + 0.25 * df["interval_score_95.mean"]
)
# df["score"] = 0.5 * df["pinball_loss.mean"] + 0.25 * (df["coverage_50.mean"] - 0.5).abs() + 0.25 * (df["coverage_95.mean"] - 0.95).abs()
df = df.sort_values("score", ascending=True)
df.head()

In [ ]:
df.head(10)

In [ ]:
# adjust names of models
df["train.bikes.selected"] = df["train.bikes.selected"].replace(
    {
        "lgbm": "LightGBM",
        "xgb-custom": "XGBoost",
        "catboost": "CatBoost",
        "quantreg": "Quantile Regression",
        "benchmark": "Benchmark",
    }
)

In [ ]:
# Create a function to format metrics as mean (±std) with highlighting
def format_table_with_metrics_and_highlighting(df, target="energy"):
    # Define display name mapping
    display_names = {
        "pinball_loss": "Pinball Loss",
        "interval_score_50": r"$\text{IS}_{0.5}$",
        "interval_score_95": r"$\text{IS}_{0.05}$",
        "coverage_50": r"$\text{Cvg}_{0.5}$",
        "coverage_95": r"$\text{Cvg}_{0.05}$",
    }

    # Create new formatted DataFrame
    formatted_table = pd.DataFrame()
    formatted_table["Model"] = df[f"train.{target}.selected"]

    # Format metrics with highlighting
    for metric in ["pinball_loss", "interval_score_50", "interval_score_95"]:
        means = df[f"{metric}.mean"]
        stds = df[f"{metric}.std"]
        min_idx = means.idxmin()

        values = []
        for idx in means.index:
            if idx == min_idx:
                values.append(f"\\textbf{{{means[idx]:.2f} (± {stds[idx]:.2f})}}")
            else:
                values.append(f"{means[idx]:.2f} (± {stds[idx]:.2f})")
        formatted_table[display_names[metric]] = values

    # Handle coverage metrics
    for metric, target in [("coverage_50", 0.5), ("coverage_95", 0.95)]:
        means = df[f"{metric}.mean"]
        stds = df[f"{metric}.std"]
        # Find index closest to target
        closest_idx = (means - target).abs().idxmin()

        values = []
        for idx in means.index:
            if idx == closest_idx:
                values.append(f"\\textbf{{{means[idx]:.2f} (± {stds[idx]:.2f})}}")
            else:
                values.append(f"{means[idx]:.2f} (± {stds[idx]:.2f})")
        formatted_table[display_names[metric]] = values

    return formatted_table


format_table_with_metrics_and_highlighting(df)

In [ ]:
formatted_table = format_table_with_metrics_and_highlighting(df)
print(
    formatted_table.to_latex(
        index=False,
        escape=False,
        caption="Results of Timeseries Cross-Validation on the Hourly Eletricity Demand Dataset. Best values are highlighted in bold.",
        label="tab:energy_results",
        position="htp",
        column_format="lrrrrr",
    )
)

In [ ]:
def format_dvc_experiments(
    exps: dict,
    target="energy",
    allow_duplicated_models=False,
    caption: str | None = None,
    label: str | None = None,
) -> tuple[pd.DataFrame, pd.DataFrame, str]:
    df = pd.DataFrame(exps)
    metrics = [col for col in df.columns if f"{target}_metrics.json" in col]
    params_col = [f"train.{target}.selected"]

    df = pd.DataFrame(exps, columns=metrics + params_col)
    df = df.dropna()
    if not allow_duplicated_models:
        df = df.drop_duplicates(subset=params_col)

    # strip the prefix in the col name until :
    df.columns = [col.split(":")[1] if ":" in col else col for col in df.columns]
    df = df.drop(columns=["avg_fit_time", "avg_pred_time"])
    df = df.sort_values("pinball_loss.mean", ascending=False, ignore_index=True)

    # adjust names of models
    df[f"train.{target}.selected"] = df[f"train.{target}.selected"].replace(
        {
            "lgbm": "LightGBM",
            "xgb-custom": "XGBoost",
            "catboost": "CatBoost",
            "quantreg": "Quantile Regression",
            "benchmark": "Benchmark",
        }
    )

    formatted_table = format_table_with_metrics_and_highlighting(df, target=target)
    # formatted_table = formatted_table.drop(
    #     columns=[r"50\% Interval Score", r"95\% Interval Score"]
    # )

    latex_code = formatted_table.to_latex(
        index=False,
        escape=False,
        caption=caption,
        label=label,
        position="htp",
        column_format="lrrrrr",
    )
    return df, formatted_table, latex_code

In [ ]:
_, _, bikes_table = format_dvc_experiments(
    exps,
    target="bikes",
    caption="Results of Timeseries Cross-Validation on the Daily Bike Count Dataset. Best values are highlighted in bold.",
    label="tab:bikes_results",
)

In [ ]:
_, _, energy_table = format_dvc_experiments(
    exps,
    target="energy",
    caption="Results of Timeseries Cross-Validation on the Hourly Electricity Demand Dataset. Best values are highlighted in bold.",
    label="tab:energy_results",
)

In [ ]:
print(bikes_table)

In [ ]:
print(energy_table)